In [8]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
df_t = pd.read_csv('aapl_triple_barrier.csv', index_col = 'Date')
df_t.head()

,days_passed,price,vert_barrier,top_barrier,bottom_barrier,out
Date,,,,,,
2000-01-06,1,0.729382,2000-01-24 00:00:00,0.833740,0.625023,1.000000
2000-01-07,2,0.763932,2000-01-25 00:00:00,0.872059,0.655804,0.905325
2000-01-10,3,0.750496,2000-01-26 00:00:00,0.835410,0.665581,1.000000
2000-01-11,4,0.712107,2000-01-27 00:00:00,0.785646,0.638568,1.000000
2000-01-12,5,0.669400,2000-01-28 00:00:00,0.734199,0.604601,1.000000


#### 4.1 Estimating the Uniqueness of a label- 3.3 결과의 t1 필요? & 멀티쓰레딩 필요?

In [3]:
def mpNumCoEvents(closeIdx, t1, molecule):
    #1 find events that span the period [molecule[0], molecule[-1]]
    t1 = t1.fillna(closeIdx[-1])
    t1 = t1[t1>=molecule[0]]
    t1 = t1.loc[:t1[molecule].max()]
    #2 count events spanninga bar
    ilco = closeIdx.searchsorted(np.array([t1.index[0], t1.max()]))
    count = pd.Series(0, index = closeIdx[iloc[0]:iloc[1] + 1])
    for tIn, tOut in t1.iteritems(): count.loc[tIn:tOut] += 1
    return count.loc[molecule[0] : t1[molecule].max()]

#### 4.2 Estimating the Average Uniqueness of a label - 3.3 결과의 t1 필요? & 멀티쓰레딩 필요?

In [4]:
def mpSampleTW(t1, numCoEvents, molecule):
    # derive average uniqueness over the event's lifespan
    wght = pd.Series(index = molecule)
    for tIn, tOut in t1.loc[wght.index].iteritems():
        wght.loc[tIn] = (1./numCoEvents.loc[tIn:tOut]).mean()
    return wght

# numCoEvents = mpPandasObj(mpNumCoEvents, ('molecule', events.index), numThreads, closeIdx=close.index, t1 = events['t1'])
# numCoEvents = numCoEvents.loc[~numCoEvents.index.duplicated(keep = 'last')]
# numCoEvents = numCoEvents.reindex(close.index).fillna(0)
# out['tW'] = mpPandasObj(mpSampleTW, ('molecule', events.index), numThreads, t1 = events['t1'], numCoEvents = numCoEvents)

#### 4.3 Build an indicator matrix

In [5]:
def getIndMatrix(barIx, t1):
    indM = pd.DataFrame(0, index = barIx, columns = range(t1.shape[0]))
    for i, (t0, t1) in enumerate(t1.iteritems()) : indM.loc[t0:t1, i] = 1
    return indM

#### 4.4 Compute Average Uniqueness

In [6]:
def getAvgUniqueness(indM):
    c = indM.sum(axis = 1)
    u = indM.div(c, axis = 0)
    avgU = u[u>0].mean()
    return avgU

#### 4.5 Return Sample from Sequential Bootstrap

In [7]:
def seqBootstrap(indM, sLength = None):
    if sLength is None : sLength = indM.shape[1]
    phi = []
    while len(phi) < sLength:
        avgU = pd.Series()
        for i in indM:
            indM_ = indM[phi+[i]]
            avgU.loc[i] = getAvgUniqueness(indM_).iloc[-1]
        prob = avgU/avgU.sum()
        phi += [np.random.choice(indM.columns, p = prob)]
    return phi

#### 4.6 Example of Sequential Bootstrip

In [10]:
def main():
    t1 = pd.Series([2,3,5], index = [0,2,4])
    barIx = range(t1.max() + 1)
    indM = getIndMatrix(barIx, t1)
    phi = np.random.choice(indM.columns, size = indM.shape[1])
    print(phi)
    print('standard uniqueness : ', getAvgUniqueness(indM[phi]).mean())
    phi = seqBootstrap(indM)
    print(phi)
    print('sequential uniquness : ', getAvgUniqueness(indM[phi]).mean())
    return 

In [11]:
main()

[1 1 1]
standard uniqueness :  0.3333333333333333
[0, 2, 2]
sequential uniquness :  0.6666666666666666


/Users/qook/Coding/Finance_ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """
/Users/qook/Coding/Finance_ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """
/Users/qook/Coding/Finance_ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


#### 4.7 Generting a Random T1 Series

In [12]:
def getRndT1(numObs, numBars, maxH):
    t1 = pd.Series()
    for i in xrange(numObs):
        ix = np.random.randint(0, numBars)
        val = ix+np.random.randint(1, maxH)
        t1.loc[ix] = val
        
    return t1.sort_index()

#### 4.8 Uniqueness from Standard and Sequential Bootstraps

In [13]:
def auxMc(numObs, numBars, maxH):
    t1 = getRndT1(numObs, numBars, maxH)
    barIx = range(t1.max() + 1)
    indM = getIndMatrix(barIx, t1)
    phi = np.random.choice(indM.columns, size = indM.shape[1])
    stdU = getAvgUniqueness(indM[phi]).mean()
    phi = seqBootstrap(indM)
    seqU = getAvgUniqueness(indM[phi]).mean()
    return {'stdU' : stdU, 'seqU' : seqU}

#### 4.10 Determination of sample weight by Absolute Return Attribution 

In [15]:
def mpSampleW(t1, numCoEvents, close, molecule):
    ret = np.log(colse).diff()
    for tIn, tOut in t1.loc[wght.index].iteritems():
        wght.loc[tIn] = (ret.loc[tIn:tOut] / numCoEvents.loc[tIn:tOut]).sum()
    return wght.abs()

#### 4.11 Implementaion of time-decay Factor

In [16]:
def getTimeDecay(tW, clfLastW = 1):
    clfW = tW.sort_index().cumsum()
    if clfLastW >=0 : slope = (1-clfLastW) / clfW.iloc[-1]
    else: slope = 1 / ((clfLastW+1) * clfW.iloc[-1])
    const = 1 - slope*clfW.iloc[-1]
    clfW = const + slope * clfW
    clfW[clfW < 0] = 0
    print(const, slope)
    return clfW